# Implementação NewHope_CPA_KEM e NewHope_CCA_PKE

### Number Theoretic Transform - CRT


http://doc.sagemath.org/html/en/reference/polynomial_rings/sage/rings/polynomial/polynomial_element_generic.html

In [1]:
class NTT(object):
#    
    def __init__(self, n=128):
        if not any([n == t for t in [32,64,128,256,512,1024,2048]]):
            raise ValueError("improper argument ",n)
        self.n = n  
        self.q = 1 + 2*n
        while True:
            if (self.q).is_prime():
                break
            self.q += 2*n
            
        self.F = GF(self.q) ;  self.R = PolynomialRing(self.F, name="w")
        w = (self.R).gen(); self.w = w
        
        g = (w^n + 1)
        xi = g.roots(multiplicities=False)[-1]
        self.xi = xi
        rs = [xi^(2*i+1)  for i in range(n)] 
        self.base = crt_basis([(w - r) for r in rs])  
    
    
    def ntt(self,f):
        def _expand_(f): 
            u = f.list()
            return u + [0]*(self.n-len(u)) 
        
        def _ntt_(xi,N,f):
            if N==1:
                return f
            N_ = N/2 ; xi2 =  xi^2  
            f0 = [f[2*i]   for i in range(N_)] ; f1 = [f[2*i+1] for i in range(N_)] 
            ff0 = _ntt_(xi2,N_,f0) ; ff1 = _ntt_(xi2,N_,f1)  
    
            s  = xi ; ff = [self.F(0) for i in range(N)] 
            for i in range(N_):
                a = ff0[i] ; b = s*ff1[i]  
                ff[i] = a + b ; ff[i + N_] = a - b 
                s = s * xi2                     
            return ff 
        
        return _ntt_(self.xi,self.n,_expand_(f))
        
    def ntt_inv(self,ff):                              ## transformada inversa
        return sum([ff[i]*self.base[i] for i in range(self.n)])
    
    def random_pol(self,args=None):
        return (self.R).random_element(args)

In [2]:
# Teste

T = NTT(2048)

f = T.random_pol(64)
# print(f)

ff = T.ntt(f)

fff = T.ntt_inv(ff)

# print(fff)
print("Correto ? ",f == fff)

Correto ?  True


In [3]:
#imports

from sage.misc.prandom import randint
import random, os
import hashlib
import numpy as np

## Implementação do NewHope_CPA_KEM

### Parâmetros necessários

In [4]:
n = 1024
q = 12289
k = 8  #noise

if n != 2^(log(n,2)) or not is_prime(q) or q < n or q%(2*n) != 1:
    raise ValueError(f"os parâmetros n={n} e q={q} não verificam as condições do esquema NewHope")
    
T = NTT(n)

In [5]:
## Os anéis usuais com o módulo ciclotómico

R_.<w> = ZZ[]
R.<x>  = QuotientRing(R_,R_.ideal(w^n+1))

Rq_.<w> = GF(q)[]
Rq.<x>  = QuotientRing(Rq_,Rq_.ideal(w^n+1))

In [6]:
# Geradores aleatorios

def random_pol(n=n):
    p = R_.random_element(n)
    return Rq(p)

def chi(k=k):
    samples = range(2*k)
    return sum([random.choice([-1,1]) for _ in samples])


def random_err(n=n):
    return Rq([chi() for _ in range(n)])

def random_seed(n):
    seed = [randint(0,1) for i in range(n)]
    seed = a2b(seed)
    
    return seed

def a2P(array):
    p = 0
    for i in array:
        p = p*x+i
    return p

def new_poly():
    
    p = lift(random_pol())
    p = list(p)
    
    return p

### Métodos auxiliares 

In [7]:
# Gerar polinómio a para gerar as chaves
def genA(seed):

    a = new_poly()
    seed = b2a(seed)
    shake = hashlib.shake_128()
    x = 200
    n1 = int(x)
    y = 168
    n2 = int(y)
    
    extseed = seed[:31]

    for i in range((n/64) - 1):
        ctr = 0
        extseed.append(i)
        extseed = a2b(extseed)
        shake.update(extseed)
        state = shake.digest(n1)
        while ctr < 64:
            buf = shake.digest(n2)
            shake.update(state)
            state = shake.digest(n1)
            j = 0
            while (j < 168 and ctr < 64):
                val = buf[j]|(buf[j+1] << 8)
                if val < 5 * q:
                    a[i*64+ctr] = val
                    ctr = ctr + 1
                j = j+2

    return a2P(a)

# Sample de polinómios
def sample(seed, nonce):
        
    shake = hashlib.shake_256()
    r = new_poly()
    seed = b2a(seed)
    x = 128
    n = int(x)

    extseed = [64]
    extseed[:31] = seed
    extseed.append(nonce)

    for i in range((n/64)-1):
        extseed.append(i)
        extseed = a2b(extseed)
        shake.update(extseed)
        buf = shake.digest(n)
        for j in range(63):
            a = buf[2 * j]
            b = buf[2 * j + 1]
            r[64*i+j]= HW(a) + q - Mod(HW(b),q)
            
    return r

# Encode da mensagem
def encode(m):

    v = lift(random_pol())
    for i in range(31):
        for j in range(7):
            mask = -((m[i]>>j)&1)
            v[8*i+j+0]   = mask&(q/2)
            v[8*i+j+256] = mask&(q/2)
            if n == 1024:
                v[8*i+j+512] = mask&(q/2)
                v[8*i+j+68]  = mask&(q/2)

    return v

# Decode da mensagem
def decode(v):

    m = []
    v = lift(v)
    
    for i in range(255):
#        t = |Mod(v[i+0],q) - (q-1)/2|
#        t = t + |Mod(v[i+256],q) - (q-1)/2|
        t = Mod(v[i+0],q) - (q-1)/2
        t = t + Mod(v[i+256],q) - (q-1)/2
        if n == 1024:
#            t = t + |Mod(v[i+512],q) - (q-1)/2|
#            t = t + |Mod(v[i+768],q) - (q-1)/2|
            t = t + Mod(v[i+512],q) - (q-1)/2
            t = t + Mod(v[i+768],q) - (q-1)/2
            t = t-q
        else:
            t = t - q/2
        t = t >> 15
        m[i>>3] = m[i>>3] or (t<<(i&7))

    return m

# Comprimir polinómio
def compress(v):

    k = 0
    t = [8]
    h = [3*n/8]
    v = lift(v)

    for l in range(n/8-1):
        i = 8*l
        for j in range(7):
            t[j] = Mod(v[i+j],q)
            t[j] = ((b2i(t[j] << 3) + q/2)/q)&7

        h[k+0] =  t[0]|(t[1]<<3)|(t[2]<<6)
        h[k+1] = (t[2] >> 2)|(t[3] << 1)|(t[4] << 4)|(t[5] << 7)
        h[k+2] = (t[5] >> 1)|(t[6] << 2)|(t[7] << 5)
        k = k+3

    return h

# Descomprimir polinómio
def decompress(a):

    k = 0
    r = lift(random_pol())

    for l in range(n/8-1):
        i = 8*l

        r[i+0] = a[k+0]&7
        r[i+1] = (a[k+0] >> 3)&7
        r[i+2] = (a[k+0] >> 6)|((a[k+1] << 2)&4)
        r[i+3] = (a[k+1] >> 1)&7
        r[i+4] = (a[k+1] >> 4)&7
        r[i+5] = (a[k+1] >> 7)|((a[k+2] << 1)&6)
        r[i+6] = (a[k+2] >> 2)&7
        r[i+7] = (a[k+2] >> 5)

        k = k+3
        for j in range(7):
            r[i+j] = (r[i+j]*q+4)>>3

    return r

# Encode do criptograma
def encodeC(u,h):

    c[0:(7*n/4-1)] = lift(u)
    c[(7*n/4-1):(7*n/4+3*n/8-1)] = h

    return c

# Decode do criptograma
def decodeC(c):

    u = lift(random_pol())
    u = c[0:(7*n/4-1)]
    h = c[(7*n/4-1):(7*n/4+3*n/8-1)]

    return (u,h)

def a2b(array):

    binary = bytearray(array)
    
    return binary

def b2a(binary):
    
    array = [d for d in binary[2:]]
    
    return array

# Hamming weight
def HW(x):
    s = 0
    x = b2a(bin(x))
        
    for i in range(len(x)):
        s = s + int(x[i])
    
    return s

### Métodos de geração de chaves, Encript e Decrypt

In [8]:
# Gerar chaves
def keyGen():

    shake = hashlib.shake_256()
    x = 64
    n = int(x)

    seed = random_seed(32)
    shake.update(seed)
    z = shake.digest(n)
    
    publicseed = z[:32]
    noiseseed  = z[32:]

    a  = genA(publicseed)
#    s  = polyBitRev(sample(noiseseed,0))
    s  = sample(noiseseed,0)
    sk = NTT(s)
#    e  = polyBitRev(sample(noiseseed,1))
    e  = sample(noiseseed,1)
    ee = NTT(e)

    b = a * sk + ee
        
    pk = (b,publicseed)

    return (pk,sk)

# Encript
def encrypt(pk,m,seed): # polyBitRev

    (b,publicseed) = pk
        
    a  = genA(publicseed)
#    s  = polyBitRev(sample(seed,0))
#    e  = polyBitRev(sample(seed,1))
    s  = sample(seed,0)
    e  = sample(seed,1)
    ee = sample(seed,2)

    t  = NTT(s)
    u  = a * t + NTT(e)
    v  = encode(m)
    vv = NTTInv(b * t) + ee + v
    h  = compress(vv)
    c = encodeC(u,h)
    
    return c

# Decript
def decrypt(c,sk):

    (u,h) = decodeC(c)
    v = decompress(h)
    m = decode(v -  NTTInv(u * sk))

    return m

### Encapsulamento 

In [9]:
def gen():
    (pk,sk) = keyGen()

    return (pk,sk)

def encapsulation(pk):

    seed = random_seed(32)
    shake = hashlib.shake_256()
    x = 64
    n = int(x)

    shake.update(seed)
    aux = shake.digest(n)
    
    k = aux[:32]

    coin = aux[32:]

    c = encrypt(pk,k,coin)

    shake.update(k)
    ss = shake.digest(n)

    return (c,ss)

def decapsulation(c,sk):

    shake = hashlib.shake_256()
    x = 32
    n = int(x)

    k = decrypt(c,sk)

    shake.update(k)
    ss = shake.digest(n)

    return ss

### Test

In [10]:
m = [random.choice([0,1]) for _ in range(n)]       # mensagem aleatória
seed = random_seed(32)

(pk,sk) = keyGen()

c = encrypt(pk,m,seed)

m1 = decrypt(c,sk)

m == m1

ValueError: ('improper argument ', [12287, 0, 12288, 3, 3, 12287, 0, 0, 0, 0, 1, 12286, 4, 12287, 12285, 1, 12288, 12286, 12285, 12286, 12285, 12288, 0, 0, 2, 12287, 5, 0, 0, 12288, 1, 12287, 12288, 12287, 12285, 3, 12288, 3, 12288, 1, 12287, 1, 12288, 12286, 1, 1, 12286, 12288, 4, 2, 2, 12287, 1, 0, 1, 12286, 0, 1, 12287, 12288, 12284, 2, 12287, 12269, 12281, 12288, 12286, 3, 12286, 12287, 3, 12288, 0, 12287, 12287, 1, 12276, 0, 59, 2, 0, 2, 1, 12288, 0, 12287, 0, 0, 0, 0, 1, 1, 0, 12287, 5, 12288, 4, 3, 5, 3, 1, 12288, 1, 0, 0, 1, 3, 12288, 19, 12288, 1, 3, 6, 1, 1, 5, 12287, 1, 1, 1, 12288, 0, 0, 0, 8, 12288, 2, 2, 1, 0, 0, 12287, 12288, 1, 0, 12288, 2, 3, 12286, 12288, 12286, 0, 12288, 3, 12179, 0, 12288, 24, 12288, 1, 0, 0, 1, 12288, 3, 12275, 3, 1, 12286, 12288, 0, 0, 12288, 12288, 0, 0, 0, 4, 1, 2, 0, 1, 12288, 0, 2, 12288, 0, 1, 1, 1, 12287, 12288, 12288, 0, 9, 12284, 0, 12286, 12278, 1, 2, 12288, 12288, 12279, 10, 2, 12287, 12288, 1, 1, 12287, 1, 12276, 12286, 12277, 12288, 1,

## Implementação do NewHope_CCA_PKE

In [12]:
def gen():
    
    shake = hashlib.shake_256()
    s = random_seed(32)
    x = 32
    n = int(x)
    
    (pk,sk) = keyGen()
    
    shake.update(pk)
    shak = shake.digest(n)
    
    sk.append(pk)
    sk.append(shak)
    sk.append(s)

    return (pk,sk)

def encapsulation(pk):

    seed = random_seed(32)
    shake = hashlib.shake_256()
    x = 32
    n = int(x)
    y = 96
    n1 = int(y)

    shake.update(seed)
    m = shake.digest(n)
    
    shake.update(pk)
    
    aux = shake.digest(n1)
    
    k    = aux[:32]
    coin = aux[32:64]
    d    = aux[64:]

    c = encrypt(pk,m,coin)
    
    cd = c.append(d)

    shake.update(cd)
    ss = shake.digest(n)

    return (cd,ss)

def decapsulation(cd,skl):
    
    seed = random_seed(32)
    shake = hashlib.shake_256()
    x = 32
    n = int(x)
    y = 96
    n1 = int(y)
    nn1 = 3*1024/8+7*1024/4
    nn2 = 7*1024/4
    
    c = cd[:nn1]
    d = cd[nn1:]
    
    sk = skl[:nn2]
    pk = skl[nn2:nn2]
    h  = skl[2*nn2:2*nn2+32]
    s  = skl[2*nn2+32:]
    
    m = decrypt(c,sk)
    
    sha = m.append(h)
    
    shake.update(sha)
    aux = shake.digest(n1)
    
    k    = aux[:32]
    dd   = aux[32:64]
    coin = aux[64:]
    
    if( c = encrypt(pk,m,coin) and d == dd):
        fail = 0
    else:
        fail = 1
        
    kappa = [2]
    
    kappa[1] = k
    kappa[2] = s
    
    shake.update(cd)
    aux = shake.digest(n)
    
    shake.update(kappa[fail].append(aux))
    ss = shake.digest(n)
    
    return ss
    

SyntaxError: invalid syntax (<ipython-input-12-4b5af115370d>, line 78)